In [83]:
import numpy as np

from hongxiongmao import download
dl = download.quandl_hxm()

import plotly.offline as py
import plotly_express as pyex

In [68]:
oecd_cli = {
    'OECD':dict(ticker='OECD/MEI_CLI_LOLITONO_OECD_M', field=[], name='Normalised (Cli), OECD Total'),
    'US':dict(ticker='OECD/MEI_CLI_LOLITONO_USA_M', field=[], name='Normalised (Cli), United States'),
    'UK':dict(ticker='OECD/MEI_CLI_LOLITONO_GBR_M', field=[], name='Normalised (Cli), United Kingdon'),
    'EURO':dict(ticker='OECD/MEI_CLI_LOLITONO_OECDE_M', field=[], name='Normalised (Cli), Europe'),
    'JAPAN':dict(ticker='OECD/MEI_CLI_LOLITONO_JPN_M', field=[], name='Normalised (Cli), Japan'),
    'AUS':dict(ticker='OECD/MEI_CLI_LOLITONO_AUS_M', field=[], name='Normalised (Cli), Australia'),
    'CANADA':dict(ticker='OECD/MEI_CLI_LOLITONO_CAN_M', field=[], name='Normalised (Cli), Canada'),
    'CHINA':dict(ticker='OECD/MEI_CLI_LOLITONO_CHN_M', field=[], name='Normalised (Cli), China'),
    'INDIA':dict(ticker='OECD/MEI_CLI_LOLITONO_IND_M', field=[], name='Normalised (Cli), India'),
    'RUSSIA':dict(ticker='OECD/MEI_CLI_LOLITONO_RUS_M', field=[], name='Normalised (Cli), Russia'),
    'BRAZIL':dict(ticker='OECD/MEI_CLI_LOLITONO_BRA_M', field=[], name='Normalised (Cli), Brazil'),
    'MEXICO':dict(ticker='OECD/MEI_CLI_LOLITONO_MEX_M', field=[], name='Normalised (Cli), Mexio'),
    'KOREA':dict(ticker='OECD/MEI_CLI_LOLITONO_KOR_M', field=[], name='Normalised (Cli), KOREA'),
    'TURKEY':dict(ticker='OECD/MEI_CLI_LOLITONO_TUR_M', field=[], name='Normalised (Cli), Turkey'),
    'ZAR':dict(ticker='OECD/MEI_CLI_LOLITONO_ZAF_M', field=[], name='Normalised (Cli), South Africa'),
    'ASIA5':dict(ticker='OECD/MEI_CLI_LOLITONO_A5M_M', field=[], name='Normalised (Cli), Major 5 Asia'),
    }

In [75]:
cli = dl.timeseries(tickers=[oecd_cli[k]['ticker'] for k in oecd_cli], start_date='-20y')
cli.columns = list(oecd_cli.keys())
cli = cli - 100
cli_dm = cli.loc[:,['US', 'UK', 'EURO', 'JAPAN', 'AUS', 'CANADA', 'KOREA', 'OECD']]
cli_em = cli.loc[:,['CHINA', 'MEXICO', 'RUSSIA', 'INDIA', 'BRAZIL', 'TURKEY', 'ZAR', 'ASIA5']]

In [122]:

def scatter_from_dataframe(df, title='OECD CLI', ytitle='', **kwargs):
    """
    Plotly Timeseries Scatter Chart - with Group
    
    Simply Plotly Scatter plot with multiple traces (lines).
    Takes pd.Dataframe & creates chart with each column as trace
    Assumption is that xaxis will be the date axis.
    
    Option to add button groups. Input is dictionary of the form
        {'NAME':['Col1', Col2]}
        Where dict key will be the button name &
        List will be the df columns shown   
    """
    
    # Colourscheme
    cmap = {0:'purple', 1:'turquoise', 2:'grey', 3:'black', 4:'lime', 5:'blue', 6:'orange', 7:'red'}
    
    ## Basic Layout Dictionary
    # Needs to come first or buttons have nowhere to append too
    layout=dict(title=title, font = {'family':'Courier New', 'size':12},
                showlegend = True, legend=dict(orientation="h"),
                margin = {'l':75, 'r':50, 'b':50, 't':50},
                yaxis={'title':ytitle, 'hoverformat':'.2f', 'tickformat':'.2f',},
                updatemenus= [dict(type='buttons', active=0, showactive = True,
                                         direction='right', pad = {'l':0, 'r':35, 't':0, 'b':0},
                                         y=-0.1, x=1,
                                         buttons=[])],
               )
    
    ## Iterate Through Each column of Dataframe to append
    data = []
    for i, c in enumerate(df): 
        data.append(dict(type='scatter', name=c, x=df.index, y=df.loc[:,c], line={'color':cmap[i], 'width':1}))

    
    ## Add Buttons
    if 'groups' in kwargs:
        for g in kwargs['groups']:
            visible = np.in1d(df.columns, kwargs['groups'][g]) 
            button= dict(label=g, method='update', args=[{'visible': visible}])
            layout['updatemenus'][0]['buttons'].append(button)    # append the button 

    fig=dict(data=data, layout=layout)
    
    return fig

cli_groups_dm = dict(ALL=list(cli_dm.columns),
                     Major=['US', 'EURO', 'UK', 'JAPAN'],
                     Other=['AUS', 'CANADA', 'KOREA', 'OECD'])

cli_groups_em = dict(ALL=list(cli_dm.columns),
                     BRIC=['CHINA', 'BRAZIL', 'RUSSIA', 'INDIA'],
                     Asia=['CHINA', 'INDIA', 'ASIA4'],
                     Latam=['MEXICO', 'BRAZIL'],
                     CEEMEA=['RUSSIA', 'TURKEY', 'ZAR'])


fig = plotly_cli(cli_dm, title='OECD Normalised CLI - Developed Markets', groups=cli_groups_dm,
                ytitle='')
py.iplot(fig)

In [ ]:
def swirlygram():
    
    
    
    return